#### Data Cleanup 2



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import timedelta
import os
from pandas_profiling import ProfileReport

In [2]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 1000

In [3]:
def generate_profile(df):
    '''generate pandas profile'''
 
    profile = ProfileReport(df)   
    profile.to_widgets()

In [4]:
#Reload data

#df = pd.read_csv('/Users/Birr/projects/cap2/data/interim/df_aged.csv', low_memory=False)
df = pd.read_csv('D:\cap\capstone2\data\interim\df_1.csv', low_memory=False)

In [ ]:
generate_profile(df)

D:\Anaconda\envs\Capstone2\lib\site-packages\pandas_profiling\visualisation\plot.py:160: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("RdBu"))
  cmap.set_bad(cmap_bad)
D:\Anaconda\envs\Capstone2\lib\site-packages\pandas_profiling\visualisation\plot.py:160: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("RdBu"))
  cmap.set_bad(cmap_bad)
D:\Anaconda\envs\Capstone2\lib\site-packages\pandas_profiling\visualisation\plot.py:160: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, 

In [ ]:
df.rename(columns={'INSTR_AME':'INSTRUCT_AME', 'INSTR_ASE':'INSTRUCT_ASE', 'INSTR_GLI':'INSTRUCT_GLI', 
                    'INSTR_IAIR':'INSTRUCT_IAIR', 'INSTR_SPRT':'INSTRUCT_SPRT', 'INSTR_NONE':'INSTRUCT_NONE',
                    'ALL-INST': 'ALL-INSTRUM', 'GLDR-INST':'GLDR-INSTRUCT', 'MAKE-INST':'MAKE-INSTRUCT',
                    'MENG-INST':'MENG-INSTRUCT', 'NGHT-INST':'NGHT-INSTRUCT', 'SENG-INST':'SENG-INSTRUCT'}, inplace=True)

In [ ]:
#after further consideration drop additional columns as they won't be needed in analysis
to_drop = ['ac_key', 'ev_id', 'find_key','second_pilot', 'ev_country', 'flt_plan_filed', 'phase_flt_spec', 'CERT_RMPT', 
           'OTHR_SUAS', 'SIMU-INST', 'SIMU-IRCV', 'SIMU-L24H', 'SIMU-L30D', 'SIMU-L90D', 'SIMU-PIC',
           'ROTO-INST', 'ROTO-IRCV', 'ROTO-L24H', 'ROTO-L30D', 'ROTO-L90D', 'ROTO-PIC', 'ROTO-TOTL',
           'OTHR_AIRS', 'OTHR_BALL', 'OTHR_GLI', 'OTHR_GYRO', 'OTHR_HELI', 'OTHR_PLFT', 'OTHR_SUAS', 'OTHR_NONE',
           'LTA-INST', 'LTA-IRCV', 'LTA-L24H', 'LTA-L30D', 'LTA-L90D', 'LTA-PIC', 'LTA-TOTL', 'Unnamed: 0', 
           'INSTR_PLFT', 'CERT_FE', 'INSTR_GYRO', 'INSTR_HELI', 'INSTR_IHEL','INSTRUM_HELI', 'INSTR_PLFT',
           'INC-NO','REL-NO', 'INSTRUM_PLFT']
df.drop(columns=to_drop, inplace=True)
df.head()

In [ ]:

to_strip = ['pilot_key', 'ev_date', 'far_part', 'crew_category', 'crew_sex', 'med_certf', 'med_crtf_vldty', 
            'cat_descript', 'sub_descript', 'sec_descript', 'subsec_descript',  'mod_descript', 'Cause_Factor',
            'ev_type', 'mid_air', 'on_ground_collision', 'light_cond', 'ev_highest_injury', 'wx_cond_basic', 'type_fly']

for col in to_strip:
    df[col] = df[col].str.strip()
    

In [ ]:
df['ev_date'] = pd.to_datetime(df['ev_date'])

In [ ]:
#analyze far_part data (type of operation)
df.far_part.value_counts()

In [ ]:
#unable to impute UNK values -- drop records
df.drop(df[df['far_part'] == 'UNK'].index, inplace = True)
df.far_part.value_counts()

In [ ]:
df['far_part'].isna().sum(axis=0)

In [ ]:
df['far_part'] = df['far_part'].fillna('delete')

In [ ]:
df.drop(df[df['far_part'] == 'delete'].index, inplace = True)

In [ ]:
df['far_part'].isna().sum(axis=0)

In [ ]:
#Analyze crew category
df['crew_category'].isna().sum(axis=0)

In [ ]:
df['crew_category'] = df['crew_category'].fillna('UNK')
df.crew_category.value_counts()

In [ ]:
unkstu = df[df['crew_category']=='UNK']
unkstu.head(10)

In [ ]:
# impute student if cert_stu = true
stud_cert = df.loc[(df['crew_category'] == 'UNK') & (df['CERT_STU'] == 1.0)]
stud_cert.head(0)

# none found

In [ ]:
#using domain knowledge will impute any UNK with less than 60 hour of flt time as Student, otherwise Pilot
unk_low_time = df.loc[(df['ALL-TOTL'] <= 60) & (df['crew_category'] == 'UNK')]
unk_low_time.shape

There are no Crew_category UNK records with less that 60 flight hours. Inpute all UNK records as pilots.

In [ ]:
df['crew_category'] = df['crew_category'].replace(['UNK'], 'PILOT')
df.crew_category.value_counts()

In [ ]:
df['crew_sex'].isna().sum(axis=0)

In [ ]:
#analyse crew_sex
df['crew_sex'] = df['crew_sex'].fillna('UNK')
df.crew_sex.value_counts()

In [ ]:
df['med_certf'].isna().sum(axis=0)

In [ ]:
df['med_certf'] = df['med_certf'].fillna('UNK')
df['med_certf'] = df['med_certf'].str.strip()
df.med_certf.value_counts()

In [ ]:
#sport pilots do now required a med cert, Basic is not an official medical certificate. Entries may have been done to reflect
#other information. Will condense SPRT/BASC to NONE.

df['med_certf'] = df['med_certf'].replace(['SPRT', 'BASC'],'NONE')
df.med_certf.value_counts()

In [ ]:
df['med_certf'].isna().sum(axis=0)

In [ ]:
df['med_crtf_vldty'].isna().sum(axis=0)

In [ ]:
df['med_crtf_vldty'] = df['med_crtf_vldty'].fillna('UNK')
df.med_crtf_vldty.value_counts()

In [ ]:
df['med_crtf_vldty'].isna().sum(axis=0)

In [ ]:
df['ev_date'].isna().sum(axis=0)

In [ ]:
class3_invalid = df.loc[(df['med_certf'] == 'CL3') & (df['med_crtf_vldty'] == 'UNK')][['pilot_key', 
                                            'ev_date', 'date_lst_med', 'med_certf', 'med_crtf_vldty']]
class3_invalid.sample(20)

Will derive medical certificate validity for UNK values by creating and checking medical expirtion date (last medical +  medical validity time) vs. event date if medical expiration is before event date then UNK will be changed to INVALID.

In [ ]:
df['date_lst_med'].isna().sum(axis=0)

In [ ]:
#replace date_last_medical NaN values with future date
df['date_lst_med'] = df['date_lst_med'].replace(np.nan, '01/01/2080')
df['date_lst_med'] = pd.to_datetime(df['date_lst_med'])
df['date_lst_med'].isna().sum(axis=0)


NaN on date last medical replaced to date in the future to indicate no information

In [ ]:
# create medical expiration date column and populate based on medical type

c1_valid_days_under40 = 365 
c1_valid_days_over40 =  int(365/2)
c2_valid_days = 365
c3_valid_days_over40  = 365 * 5
c3_valid_days_under40 = 365 * 2

med_cert_idx = df.columns.get_loc('med_certf')
age_idx = df.columns.get_loc('crew_age')
df.insert(9,'medical_exp_date', '')
df['medical_exp_date'] = pd.to_datetime(df['medical_exp_date'])
last_med_idx = df.columns.get_loc('date_lst_med')

for i in range (len(df)):
    if (df.iloc[i, med_cert_idx] == 'CL1') & (df.iloc[i, age_idx] >= 40):
        df.iloc[i, 9] = df.iloc[i, last_med_idx] + timedelta(c1_valid_days_over40)
    elif (df.iloc[i, med_cert_idx] == 'CL1') & (df.iloc[i, age_idx] < 40):
        df.iloc[i, 9] = df.iloc[i, last_med_idx] + timedelta(c1_valid_days_under40)
    elif (df.iloc[i, med_cert_idx] == 'CL2'): 
        df.iloc[i, 9] = df.iloc[i, last_med_idx] + timedelta(c2_valid_days)
    elif (df.iloc[i, med_cert_idx] == 'CL3') & (df.iloc[i, age_idx] >= 40):
        df.iloc[i, 9] = df.iloc[i, last_med_idx] + timedelta(c3_valid_days_over40)
    elif (df.iloc[i, med_cert_idx] == 'CL3') & (df.iloc[i, age_idx] < 40):
        df.iloc[i, 9] = df.iloc[i, last_med_idx] + timedelta(c3_valid_days_under40)               

In [ ]:
unk = df[df['med_crtf_vldty'] == 'UNK']


ev_idx = unk.columns.get_loc('ev_date')
exp_idx = unk.columns.get_loc('medical_exp_date')
val_idx = unk.columns.get_loc('med_crtf_vldty')

for i in range (len(unk)):
    if unk.iloc[i, ev_idx] > unk.iloc[i, exp_idx]:
        unk.iloc[i, val_idx] = 'INVALID'

        
unk.med_crtf_vldty.value_counts()     

In [ ]:
# subset newly imputed invalid certificates and backfill df

inval = unk[unk['med_crtf_vldty'] == 'INVALID']

val_idx = df.columns.get_loc('med_crtf_vldty')
for i in range (len(inval)):
    for r in range (len(df)):
        if inval.iloc[i,0] == df.iloc[r,0]:
            df.iloc[r, val_idx] = 'INVALID'
           

In [ ]:
df.med_crtf_vldty.value_counts()

In [ ]:
#combined.to_csv('/Users/Birr/projects/cap2/data/interim/df_aged.csv', index=False)
df.to_csv('D:\cap\capstone2\data\interim\df_2.csv', index=False)

In [ ]:
#Reload data

#df = pd.read_csv('/Users/Birr/projects/cap2/data/interim/df_aged.csv', low_memory=False)
df = pd.read_csv('D:\cap\capstone2\data\interim\df_2.csv', low_memory=False)
df.head()


In [ ]:
to_drop = ['Cause_Factor', 'ev_type', 'mid_air', 'on_ground_collision', 'ev_highest_injury']
df.drop(to_drop, axis=1, inplace=True)
df.head()

In [ ]:
# analyze weather condition

In [ ]:
df.wx_cond_basic.unique()

In [ ]:
df.wx_cond_basic.isna().sum(axis=0)

In [ ]:
df['wx_cond_basic'] = df['wx_cond_basic'].fillna('UNK')

In [ ]:
df.wx_cond_basic.value_counts()

In [ ]:
# analyze type fly
df.type_fly.value_counts()

In [ ]:
#condense categories & adjust entries for readability

rep_dict = {'PUBU':'PUBLIC', 'PUBL':'PUBLIC', 'PUBF':'PUBLIC', 'PUBS':'PUBLIC', 'FERY':'FERRY', 'POSI':'FERRY', 'BUS':'BUSS',
            'EXEC':'BUSS', 'OWRK':'OTHRWORK', 'INST':'INSTRUC', 'AOBV':'AERIALOBS', 'FLTS':'FLTTEST', 'ASHO':'ASHOW',
            'GLDT':'GLDTTOW', 'BANT':'BANNTOW', 'AAPL':'CROPDUST', 'ADRP':'AIRDROP', 'EXLD':'EXTERNALLOAD'}

df.type_fly = df.type_fly.replace(rep_dict)
df.type_fly.value_counts()

In [ ]:
#combined.to_csv('/Users/Birr/projects/cap2/data/interim/df_aged.csv', index=False)
df.to_csv('D:\cap\capstone2\data\interim\df_3.csv', index=False)